# Scraping Zip codes data from Wikipedia

**This work sheet is to scarpe Cananda Zip code data from Wikipedia**

In [1]:
! pip install lxml 

     |████████████████████████████████| 5.8MB 26.4MB/s eta 0:00:01


In [2]:
import requests
import lxml.html as lh
import pandas as pd

## scrape the table cell from the website

In [87]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [68]:
T = tr_elements[1].text_content()
T

'\nM1A\nNot assigned\nNot assigned\n'

In [69]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

## Parse table header

In [88]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content().strip('\n')
    print(i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood


## Create Pandas Data Frame

In [89]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    temp = tr_elements[j]
    temp = temp.text_content()
    # Ignore cells with a borough that is Not assigned.
    if temp[5:17] != 'Not assigned':
        T=tr_elements[j]  #T is our j'th row
 
        #If row is not of size 3, the //tr data is not from our table 
        if len(T)!=3:
            break

        #i is the index of our column
        i=0
        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content().strip('\n')
            #print(data+'**')
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1

In [72]:
[len(C) for (title,C) in col]

[210, 210, 210]

In [90]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [74]:
df.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [91]:
df.shape

(210, 3)

## Concentrate Neighbourhood column grouped by the Postcode and Borough

In [92]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [93]:
df.shape

(103, 3)